# Gaze Tracker — Quickstart (RU)

Этот ноутбук — шпаргалка по проекту: что он делает, как запускать локально/в Docker, как калибровать и как дергать API.

**Цель модели/сервиса:** определить направление взгляда пользователя на мониторе и визуализировать результат.

> Проект **не обучается на собственном датасете**. Используется предобученная нейросеть для лица/ориентиров + эвристики по зрачку + персональная калибровка.


## 1) Что внутри

- `gaze_tracker/detectors/yunet.py` — YuNet (ONNX) через OpenCV: bbox лица + 5 ориентиров (глаза/нос/рот).
- `gaze_tracker/vision/pupil.py` — эвристический поиск центра зрачка внутри crop'ов глаз.
- `gaze_tracker/calibration.py` — персональная калибровка: ridge‑регрессия `features -> (x_norm, y_norm)`.
- `gaze_tracker/service.py` — пайплайн кадра: YuNet → глаза → зрачок → признаки → (калиброванные координаты).
- `gaze_tracker/api.py` — FastAPI + Web UI, эндпоинты `/api/*`.

**Web UI** живёт в `gaze_tracker/web/` и работает через API на `localhost`.

## 2) Быстрый запуск (локально)

Открой терминал в корне проекта и выполни:

```powershell
python -m pip install -r requirements.txt
python -m gaze_tracker --port 8000
```

Открыть UI: `http://127.0.0.1:8000/`

Примечания:
- На первом запуске сервис может скачать YuNet‑модель (ONNX) в кэш пользователя.
- Если порт занят (например `winerror 10048`) — запускай на другом порту: `--port 8010`.

## 3) Калибровка (9 точек)

1) Нажми `Start`

2) Нажми `Calibrate (9 pts)` — откроется полноэкранный оверлей

3) На каждой точке: смотри на оранжевую точку и нажимай `Capture` (можно `Space`)

4) После 9 точек нажми `Fit`

После калибровки:
- стрелка направления остаётся на видео (debug),
- **точка на «экране»** начинает двигаться по координатам `screen.x_norm/screen.y_norm`.

Советы по качеству:
- свет/контраст важны (камера + зрачок),
- старайся держать голову примерно в одном месте во время калибровки,
- если `warnings: ["no_pupil"]` — попробуй другой угол/свет, открой глаза шире.

## 4) Docker

### Build + Run

```powershell
docker build -t gaze-tracker .
docker run --rm -p 8000:8000 gaze-tracker
```

UI: `http://localhost:8000/`

### Docker Compose

```powershell
docker compose up --build
```

Compose создаёт volume для кэша модели (`gaze-tracker-cache`).

## 5) API (эндпоинты)

- `GET  /api/health`
- `POST /api/predict` (multipart: `file`)
- `POST /api/calibration/reset`
- `POST /api/calibration/capture` (multipart: `file`, `target_x`, `target_y`)
- `POST /api/calibration/fit`
- `GET  /api/calibration/status`


In [ ]:
# Проверка установленных версий (полезно для отчёта/повторяемости)
import importlib.metadata as m

pkgs = ["numpy", "opencv-python", "fastapi", "uvicorn", "python-multipart"]
for name in pkgs:
    print(f"{name}: {m.version(name)}")


In [ ]:
# Healthcheck из Python (работает, если сервер уже запущен в отдельном терминале)
import json
import urllib.request

url = "http://127.0.0.1:8000/api/health"
try:
    with urllib.request.urlopen(url, timeout=3) as r:
        data = r.read().decode("utf-8")
    print(json.loads(data))
except Exception as e:
    print("Не удалось подключиться к серверу.")
    print("Запусти в терминале: python -m gaze_tracker --port 8000")
    print("Ошибка:", e)


In [ ]:
# Пример /api/predict (удобнее через requests). Опционально: установи requests.
# ВАЖНО: это пример для отправки уже сохранённого кадра (JPEG/PNG) в API.

import importlib.util

if importlib.util.find_spec("requests") is None:
    print("requests не установлен.")
    print("Установи (опционально): python -m pip install requests")
else:
    import requests

    img_path = r"PATH_TO_IMAGE.jpg"  # TODO: укажи путь к файлу изображения
    url = "http://127.0.0.1:8000/api/predict"

    with open(img_path, "rb") as f:
        r = requests.post(url, files={"file": ("frame.jpg", f, "image/jpeg")}, timeout=10)
    print("status:", r.status_code)
    print(r.json())


## 6) Где хранится калибровка

По умолчанию файл калибровки сохраняется в кэш пользователя:

- Windows: `%LOCALAPPDATA%\\gaze_tracker\\state\\calibration_v2.npz`
- Linux/macOS: `$XDG_CACHE_HOME/gaze_tracker/state/calibration_v2.npz` (или `~/.cache/...`)

Можно переопределить:

```powershell
python -m gaze_tracker --calib-path .\calibration.npz
```

И загрузить при старте:

```powershell
python -m gaze_tracker --load-calibration
```

## 7) Полезные команды

Показать параметры запуска:

```powershell
python -m gaze_tracker --help
```

Если хочешь поставить пакет как CLI:

```powershell
python -m pip install -e .
gaze-tracker --port 8000
```

## 8) Траблшутинг

**Порт занят (`winerror 10048`)**
```powershell
python -m gaze_tracker --port 8010
```

**Камера не даётся браузеру**
- проверь разрешения на камеру,
- открывай `http://127.0.0.1:8000/` или `http://localhost:8000/`.

**Точка «не двигается»**
- до калибровки видно только **стрелку направления**,
- точка на «экране» появляется после `Calibrate → Capture ×9 → Fit`.

**Точность слабая**
- улучшай свет, держи голову стабильнее,
- повтори калибровку (Reset calib → Calibrate).
